In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 55.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6d45448e139daf4d0cbeb87bcb5fc11c70e2955c9d14015c883d7f33d1820bf4
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark import SparkConf, SparkContext
import csv
spark_conf = SparkConf()\
        .setAppName("Week 6 lab solution")
sc=SparkContext.getOrCreate(spark_conf) 

In [6]:
input_path = 'file:///content/drive/MyDrive/comp5349/'
movie_file = input_path + "movies.csv"
text_file = input_path + "1984_processed.txt"
#movieData = sc.textFile(movie_file)

#### Q1. Find the top 5 bigrams in 1984 preprocessed text

In [2]:
def bigram(line):
    """This function converts line of input text into word level bigram token 
   
    Args:
        record (str): A line of input text file
    Returns:
        a list of word level bigram represented by tuple
    """
    
    words = line.strip().split(" ")
    if len(words) < 2:
        return []
    bigrams = zip(words[:-1], words[1:])
    results =[]
    for bigram in bigrams:
        results.append((bigram,1))
    return results

#test bigram function
line = "wrote big brother, whether refrained writing it, made"
bigram(line)

[(('wrote', 'big'), 1),
 (('big', 'brother,'), 1),
 (('brother,', 'whether'), 1),
 (('whether', 'refrained'), 1),
 (('refrained', 'writing'), 1),
 (('writing', 'it,'), 1),
 (('it,', 'made'), 1)]

In [8]:
text_rdd = sc.textFile(text_file)

bigram_rdd = text_rdd.flatMap(bigram) 
bigram_counts= bigram_rdd.reduceByKey(lambda a, b: a + b)
bigram_counts.sortBy(lambda r: r[1],ascending=False).take(5)

[(('big', 'brother'), 67),
 (('said', 'winston'), 43),
 (('old', 'man'), 38),
 (('thought', 'police'), 38),
 (('said', "o'brien"), 37)]

#### Q2. Find out the year with most Sci-Fi movies released.

In [10]:
genre="Sci-Fi"
def filterGenreYear(record):
    """ This function filters all movies in  genre  
        and returns a tuple (year, 1), 
        indicating 1 sci-fi movie in this year.
    """
    for row in csv.reader([record]):
        if len(row) != 3:
            return []
        if genre in row[2]:
            title = row[1].strip()
            year = title[len(title)-5:len(title)-1]
            return [(year,1)]
        else:
            return []
# check function output
record = "111235,Jodorowsky's Dune (2013) ,Documentary|Sci-Fi"
filterGenreYear(record)

[('2013', 1)]

In [12]:
from operator import add
movieData = sc.textFile(movie_file)
movieData.flatMap(filterGenreYear) \
    .reduceByKey(add) \
    .sortBy(lambda r: r[1],ascending=False) \
    .take(1)

[('2009', 49)]